In [ ]:
from cqapi.user_editor import Conquery, ConqueryConnection
eva = Conquery()

eva.conn = ConqueryConnection(url="https://eva.ingef.de", token=open("./fdb_token").read())

In [ ]:
query = eva.from_existing_query("Diabetes-Bayern-60", get_original=True)

In [ ]:
eva.get_data(query)

In [ ]:
query.show_json()

In [ ]:
eva.search_concept("icd", "diabetes")

In [ ]:
from cqapi.queries.base_elements import create_query
features= [
    create_query(concept_id="fdb_destatis.ambulante_kosten", concepts=eva.concepts.concepts),
    create_query(concept_id="fdb_destatis.arzneimittelkosten_brutto", concepts=eva.concepts.concepts),
    create_query(concept_id="fdb_destatis.krankenhauskosten", concepts=eva.concepts.concepts)
]
export_query = eva.new_absolute_export_query(editor_query=eva.conn.execute_query(query),
                                             start_date="01.01.2018",
                                             end_date="31.12.2021",
                                             features=features,resolution="YEARS")

data = eva.get_data(export_query)
data.head()

In [ ]:
import pandas as pd

data=data[data["Zeiteinheit"] == "Jahr"]
data["Jahr"] = data["Zeitraum"].apply(lambda x: x["min"].year)

data_to_plot = pd.concat([data[["Jahr", cost_col]].assign(Bereich=cost_col).rename(columns={cost_col:"Kosten"}) 
                          for cost_col in ["Arzneimittelkosten (Brutto)", "Ambulante Kosten", "Krankenhauskosten"]])

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
sns.barplot(data=data_to_plot, x="Jahr", y="Kosten", hue="Bereich", palette="Reds")